# Trabalho 1 - Nivelamento

Considere os datasets abaixo, estratégias de pré-processamento, medidas de avaliação, métodos de comparação estatística e os seguintes algoritmos de aprendizado de máquina: árvore de decisão, random forest e k-nearest neighbor. A partir disso, responda as seguintes perguntas:

1. Qual o algoritmo de AM mais adequado para cada problema?
2. Qual o algoritmo de AM mais adequado para todos os problemas?

Para responder essas questões construa um notebook no colab ou um ambiente similar. Documente de forma clara cada passo e justifique suas decisões.

In [1]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.patches as mpatches
from scipy.io import arff
import matplotlib as plt
import urllib.request
import pandas as pd
import numpy as np
import io

In [2]:
PhishingWebsites_link = "https://www.openml.org/data/download/1798106/phpV5QYya"
arrhythmia_link       = "https://www.openml.org/data/download/53551/arrhythmia.arff"
Satellite_link        = "https://www.openml.org/data/download/16787463/phpZrCzJR"
airlines_link         = "https://www.openml.org/data/download/66526/phpvcoG8S"
AedesSex_link         = "https://github.com/denismr/Classification-and-Counting-with-Recurrent-Contexts/raw/master/codeAndData/data/AedesSex.csv"
phoneme_link          = "https://www.openml.org/data/download/1592281/php8Mz7BG"
adult_link            = "https://www.openml.org/data/download/1595261/phpMawTba"

In [3]:
def loadDataset_Arff(url: str) -> pd.DataFrame:
  ftpstream = urllib.request.urlopen(url)
  return pd.DataFrame(arff.loadarff(io.StringIO(ftpstream.read().decode('utf-8')))[0])


In [57]:
def get_dummies(df: pd.DataFrame, col:str) -> pd.DataFrame:
  return pd.get_dummies(df,prefix=col,prefix_sep='.',columns=[col]).copy()

In [5]:
def select_nominals(dataset: pd.DataFrame) -> list:
    """
    Seleciona os atributos não numéricos do dataset.
    """
    att: list = []
    for i, v in dataset.dtypes.items():
        if(v == "object"):
            att.append(i)
    return att

def is_binary_nominal(nominals: list) -> "tuple[bool, list[str]]":
    """
    Retorna se uma lista de atributos nominais contém apenas 2 valores distintos ou não.
    """
    unique_att: "list[str]" = []
    for n in nominals:
        if(not n in unique_att):
            unique_att.append(n)
    return len(unique_att) == 2, unique_att

In [6]:
def convert_binary_nominal_to_numeric(dataset: pd.DataFrame, attribute: str) -> pd.DataFrame:
    """
    Converte atributos nominais binários de um dataset para numéricos binários (0 ou 1).
    """
    new_dt = dataset.copy()
    attributes = new_dt[attribute]
    unq_attr: dict['str', int] = {}
    # Pegando os atributos únicos
    i = 0
    for a in attributes:
        if(not a in unq_attr):
            unq_attr[a] = i
            i += 1
    # Substituindo os valores nominais por numéricos.
    for u in unq_attr:
        new_dt.loc[new_dt[attribute] == u, attribute] = unq_attr[u]
    return new_dt

In [7]:
def convert_nominal_to_numeric(dataset: pd.DataFrame) -> pd.DataFrame:
    """
    Converte todos os atributos nominais de um dataset para atributos numéricos.
    """
    nominals = select_nominals(dataset)
    new_dt = dataset.copy()
    for att in nominals:
        binary, values = is_binary_nominal(new_dt[att])
        if(binary): # Atributo nominal binário
            new_dt = convert_binary_nominal_to_numeric(new_dt, att)
        else: # Atributo nominal não binário
            new_dt = get_dummies(new_dt, att)
    return new_dt

In [83]:
def proportion(Dataset,target="Target"):
  target_count = Dataset[target].value_counts()
  print('Class 0:', target_count[0])
  print('Class 1:', target_count[1])
  print('Proporção:', round(target_count[1] / target_count[0], 2), ': 1')
  target_count.plot(kind='bar', title='Count (target)',color = ['#1F77B4', '#FF7F0E']);

In [88]:
def normalize(dataset) -> pd.DataFrame:
  # Separação dos dados preditivos e dos valores
  X = dataset.drop(['Target'], axis=1).copy()
  Y = dataset['Target'].copy()

  # Normalização
  scaler = MinMaxScaler()
  X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
  dataset = X
  dataset["Target"] = Y
  return dataset

# Lendo todos os datasets

In [89]:
Dataset_PhishingWebsites = loadDataset_Arff(PhishingWebsites_link)
Dataset_arrhythmia = loadDataset_Arff(arrhythmia_link)
Dataset_Satellite = loadDataset_Arff(Satellite_link)
Dataset_airlines = loadDataset_Arff(airlines_link)
Dataset_AedesSex = pd.read_csv(AedesSex_link)
Dataset_phoneme = loadDataset_Arff(phoneme_link)
Dataset_adult = loadDataset_Arff(adult_link)

Abaixo será feito a conversão das classes targets para True e False, e trocado o nome da coluna para Target para as que não estão. Também será feito uma analise de distribuição e procura de dados nominais.

In [90]:
Dataset_PhishingWebsites['Target'] = [True if x == '-1'.encode() else False for x in Dataset_PhishingWebsites['Result']]
Dataset_PhishingWebsites = Dataset_PhishingWebsites.drop(['Result'],axis=1)
# Dataset_PhishingWebsites.info()
# proportion(Dataset_PhishingWebsites)

In [91]:

Dataset_arrhythmia['Target'] = [True if x == 'P'.encode() else False for x in Dataset_arrhythmia['binaryClass']]
Dataset_arrhythmia = Dataset_arrhythmia.drop(['binaryClass'],axis=1)
# Dataset_arrhythmia
# proportion(Dataset_arrhythmia)

In [92]:
##Sem nominais usando a função select_nominals
Dataset_Satellite['Target'] = [True if x == 'Anomaly'.encode() else False for x in Dataset_Satellite['Target']]
# proportion(Dataset_Satellite)


In [ ]:
##Dataset com muitos dados nominais, para aplicar o knn será necessário usar a função getdummies em AirportFrom e AirportTo.
Dataset_airlines['Target'] = [True if x == '1'.encode() else False for x in Dataset_airlines['Delay']]
Dataset_airlines = Dataset_airlines.drop(['delay'],axis=1)
# Dataset_airlines = get_dummies(Dataset_airlines,'AirportFrom')
# Dataset_airlines = get_dummies(Dataset_airlines,'AirportTo')
# proportion(Dataset_airlines)
# Dataset_airlines

In [95]:
Dataset_AedesSex['Target'] = [True if x == 'F' else False for x in Dataset_AedesSex['sex']]
Dataset_AedesSex = Dataset_AedesSex.drop(['sex'],axis=1)
# proportion(Dataset_AedesSex)
# Dataset_AedesSex


In [96]:
##Dataset desbalanceado, analisar como se comporta nos algoritmos de aprendizagem para saber se é necessário balancear.
Dataset_phoneme['Target'] = [True if x == '1'.encode() else False for x in Dataset_phoneme['Class']]
Dataset_phoneme = Dataset_phoneme.drop(['Class'],axis=1)
# proportion(Dataset_phoneme)
# Dataset_phoneme

In [97]:
##Dataset com muitas colunas nominais, será necessário gerar o dummie das mesmas.
Dataset_adult['Target'] = [True if x == '<=50K'.encode() else False for x in Dataset_adult['class']]
Dataset_adult = Dataset_adult.drop(['class'],axis=1)
# Dataset_adult

In [ ]:
## NOR_Dataset_PhishingWebsites = normalize(Dataset_PhishingWebsites)
## NOR_Dataset_arrhythmia = normalize(Dataset_arrhythmia)
## NOR_Dataset_Satellite = normalize(Dataset_Satellite)
## NOR_Dataset_AedesSex = normalize(Dataset_AedesSex)
## NOR_Dataset_phoneme = normalize(Dataset_phoneme)

##Para normalizar esses datasets precisará gerar os dumies
## # NOR_Dataset_airlines = normalize(Dataset_airlines)
## # NOR_Dataset_adult = normalize(Dataset_adult)

